## **Project Name**    -   Video Games Sales & Engagement Analysis

#### **Project type** - Exploratory data analysis and Business intelligence dashboard
##### **Contribution**    - Individual
##### **Author**          - Bhavesh Kumar

## **Project Summary**

The goal of this project is to perform exploratory data analysis (EDA) on video game engagement and sales datasets to understand industry trends, user behavior patterns, genre dynamics, and regional sales performance. The analysis focuses on identifying the key factors that influence commercial success by integrating user engagement metrics (ratings, wishlist, plays) with global and regional sales data.

### **Data Collection and Cleaning:**

- Game Engagement Dataset (ratings, wishlist, plays, reviews, genres, release details)
- Video Game Sales Dataset (platform, publisher, regional sales, global sales)

##### *Data cleaning steps included:*
- Handling missing values in release dates, ratings, and team fields
- Removing duplicate records
- Exploding multi-genre entries for detailed genre-level analysis
- Exploding multi-genre entries for detailed genre-level analysis
- Ensuring proper data types for year and numerical columns

### **Exploratory Data Analysis and Visualization:**
 
Univariate, bivariate, and multivariate analysis was conducted using Python libraries (matplotlib and seaborn) and SQL queries.

### **Problem Statement**

##### The goal of this project is to identify the key factors that influence video game sales by analyzing production volume, genre trends, user engagement behavior, and regional market performance. The findings aim to provide insights into which types of games and markets generate the highest commercial returns.

### **Insights:**

- North America is the dominant revenue driver, followed by Europe, with Japan showing strong specialization in Role-Playing games.
- The industry peaked around 2008–2009, reflecting major console generation cycles.
- Wishlist count shows a moderate positive relationship with global sales, suggesting pre-release demand can indicate commercial success.
- Action and RPG genres dominate both production and revenue, while MOBA and Shooter genres exhibit the highest engagement intensity.
- Regional preferences differ significantly, with Western markets favoring Shooter and Sports genres, while Japan prefers RPG titles.
- High production volume does not necessarily translate to high engagement or sales, highlighting differences between supply concentration and market performance.
- The gaming industry follows a blockbuster-driven model, where a small number of titles generate disproportionately high revenue.
- Engagement and quality (ratings) do not always directly correlate with total sales, indicating the impact of brand equity and marketing.

### **Business Impact**

This analysis provides actionable insights for game developers, publishers, and investors by identifying the key drivers of commercial success in the video game industry. By linking user engagement metrics such as ratings, wishlist counts, and play frequency with regional and global sales performance, the project highlights which genres, platforms, and markets generate the highest returns. The findings support data-driven decision-making in areas such as genre investment strategy, regional market targeting, platform prioritization, and pre-release demand forecasting. Overall, the integrated engagement and sales analysis enables stakeholders to better align production, marketing, and distribution strategies to maximize revenue and long-term player engagement.

### **Starting with the EDA**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Loading the datasets
games_df = pd.read_csv('Datasets/games.csv')
vgsales_df = pd.read_csv('Datasets/vgsales.csv')

### Preprocessing games dataset

In [ ]:
# Structure of the games dataset
games_df.shape

In [ ]:
games_df.head(3)

In [ ]:
games_df.info()

In [ ]:
# Removing columns that are not needed for analysis
games_df.drop(columns=['Unnamed: 0','Summary','Reviews'], inplace=True)

In [ ]:
# All column names
games_df.columns

In [ ]:
# renaming the column names
games_df.columns = games_df.columns.str.lower().str.replace(' ', '_')

In [ ]:
games_df = games_df.rename(columns={'genres': 'genre'})

In [ ]:
# Keep the original column as-is
games_df['release_date_raw'] = games_df['release_date']  

# Create a new column with converted datetime values
games_df['release_date'] = pd.to_datetime(games_df['release_date'], errors='coerce')

# New 'year' column
games_df['year'] = games_df['release_date'].dt.year

In [ ]:
# Show all rows where release_date is null (NaT)
missing_dates = games_df['release_date'].isna()
games_df.loc[missing_dates]

In [ ]:
games_df.drop(columns=['release_date_raw'], inplace=True)

In [ ]:
games_df['team'] = games_df['team'].astype(str).str.strip("[]").str.replace("'", "").str.replace('"', '')

In [ ]:
cols = ['times_listed', 'number_of_reviews', 'plays', 'playing', 'backlogs', 'wishlist']

for col in cols:
    games_df[col] = games_df[col].str.replace(',', '').apply(
        lambda x: float(x.replace('K', '')) * 1000 if 'K' in str(x) else float(x))

In [ ]:
games_df.head(3)

In [ ]:
games_df['genre'] = games_df['genre'].str.split(',')
games_df = games_df.explode('genre').reset_index(drop=True)
games_df['genre'] = games_df['genre'].str.strip('[]').str.strip("'").str.strip().str.title()
games_df['genre'] = games_df['genre'].replace('', np.nan)

In [ ]:
games_df.head()

In [ ]:
games_df.info()

In [ ]:
games_df.isnull().sum()

In [ ]:
games_df['rating'] = games_df['rating'].fillna(games_df['rating'].median())

In [ ]:
games_df.isnull().sum()

In [ ]:
games_df.head(3)

In [ ]:
float_cols = ['times_listed', 'number_of_reviews', 'plays', 'playing','backlogs', 'wishlist','year']
for col in float_cols:
    games_df[col] = pd.to_numeric(games_df[col], errors='coerce').astype('Int64')

In [ ]:
# Define the desired order of columns
new_order = [
    'title', 'genre', 'release_date', 'year', 'team', 'rating',
    'times_listed', 'number_of_reviews', 'plays', 'playing',
    'backlogs', 'wishlist'
]
# Reorder DataFrame
games_df = games_df[new_order]

In [ ]:
games_df.head()

In [ ]:
print(games_df.duplicated().sum())
games_df.drop_duplicates(inplace=True)

In [ ]:
games_df.info()

In [ ]:
games_df.describe()

In [ ]:
games_df.to_csv('Datasets/games_cleaned.csv', index=False)

### ________________________________________________________________________________________________________________________________________________________________________ ###

### Preprocessing sales dataset

In [ ]:
vgsales_df.head()

In [ ]:
vgsales_df.shape

In [ ]:
vgsales_df = vgsales_df.drop(columns=['Rank'])
vgsales_df.columns = vgsales_df.columns.str.lower().str.replace(' ', '_').str.strip()

In [ ]:
vgsales_df.columns

In [ ]:
vgsales_df.info()

In [ ]:
vgsales_df['year'] = pd.to_numeric(vgsales_df['year'], errors='coerce').astype('Int64')

In [ ]:
text_cols = ['name', 'platform', 'genre', 'publisher']
for col in text_cols:
    vgsales_df[col] = vgsales_df[col].str.strip().str.replace('.', '').str.replace("'", '')

In [ ]:
vgsales_df.head(10)

In [ ]:
vgsales_df['platform'].unique()

In [ ]:
# Mapping dictionary: platform → brand
platform_category = {
    # Play_Station
    "PS": "Play_Station", "PS2": "Play_Station", "PS3": "Play_Station", "PS4": "Play_Station",
    "PSP": "Play_Station", "PSV": "Play_Station",
    # Xbox
    "XB": "Xbox", "X360": "Xbox", "XOne": "Xbox",
    # Nintendo
    "NES": "Nintendo", "SNES": "Nintendo", "N64": "Nintendo", "GC": "Nintendo",
    "Wii": "Nintendo", "WiiU": "Nintendo", "GB": "Nintendo", "GBA": "Nintendo",
    "DS": "Nintendo", "3DS": "Nintendo",
    # Sega
    "GEN": "Sega", "SAT": "Sega", "DC": "Sega", "SCD": "Sega", "GG": "Sega",
    # NEC
    "PCFX": "NEC", "TG16": "NEC",
    # Bandai 
    "WS": "Bandai",
    # Panasonic
    "3DO": "Panasonic",
    # SNK
    "NG" : "SNK",
    # Atari
    "2600": "Atari",
    # PC
    "PC": "PC"
}

# Add new column with brand flag
vgsales_df["platform_category"] = vgsales_df["platform"].map(platform_category)
vgsales_df.head(10)

In [ ]:
vgsales_df.isna().sum()

In [ ]:
vgsales_df = vgsales_df.drop_duplicates()

In [ ]:
vgsales_df.info()

In [ ]:
vgsales_df.describe()

In [ ]:
vgsales_df.to_csv('Datasets/vgsales_cleaned.csv', index=False)

### _________________________________________________________________________________________________________________________________________________________________________ ###

### Merging the datasets

In [ ]:
merged_data = pd.merge(games_df, vgsales_df, left_on = 'title', right_on = 'name', how='inner')

In [ ]:
merged_data.shape

In [ ]:
merged_data['platform'].unique()

### _________________________________________________________________________________________________________________________________________________________________________ ###

### Creating Visuals to get insights

**1.What are the top-rated games by user reviews?**

In [ ]:
top_rated = (games_df.groupby('title').agg(avg_rating=('rating', 'mean'),
         total_reviews=('number_of_reviews', 'sum'))
    .sort_values('avg_rating', ascending=False)
    .head(10)
)
top_rated_sorted = top_rated.sort_values('avg_rating')

plt.figure(figsize=(8,6))
bars = plt.barh(
    top_rated_sorted.index,
    top_rated_sorted['avg_rating']
)
plt.xlabel("Average Rating")
plt.title("Top 10 Highest Rated Games")
for i, v in enumerate(top_rated_sorted['avg_rating']):
    plt.text(v + 0.02, i, f"{v:.2f}")

plt.show()

#### 📝 Insights:

- #### The top-rated games are tightly clustered within a narrow rating band (4.6–4.8), suggesting that once games reach an elite quality threshold, differences in perceived quality become marginal.
- #### Elden Ring’s expansion leads the ranking, indicating strong sustained franchise quality and successful post-launch content strategy.

**2.What are the most common genres?**

In [ ]:
genre_counts = games_df['genre'].value_counts().head(10)

plt.figure()
genre_counts.plot(kind='bar')
plt.title("Top 10 Most Common Genres")
plt.xlabel("Genre")
plt.ylabel("Games Count")
plt.xticks(rotation=45)
plt.show()

#### 📝 Insights:
- ##### Adventure is the most produced genre in the dataset, suggesting that developers favor narrative-driven and exploration-based game design.
- ##### Although Adventure dominates production volume, it does not necessarily lead in engagement intensity or revenue, highlighting the difference between supply concentration and market performance.

**3.What is the game release trend over years?**

In [ ]:
release_trend = (
    games_df
    .groupby('year')['title']
    .nunique()
    .sort_index()
)

plt.figure()
release_trend.plot(kind='line')
plt.title("Game Releases Over Time")
plt.xlabel("Year")
plt.ylabel("Number of Releases")
plt.show()

#### 📝 Insights:
- ##### The gaming industry has experienced consistent long-term expansion, with production accelerating significantly after the early 2000s.
- ##### The sharp decline in the final year likely reflects incomplete dataset coverage rather than a true market contraction.

**4.What are the top 10 most wishlisted games?**

In [ ]:
top_wishlist = (
    games_df
    .groupby('title', as_index=False)
    .agg(total_wishlist=('wishlist', 'mean'))
    .sort_values('total_wishlist', ascending=False)
    .head(10)
)
plt.figure(figsize=(8,6))
plt.barh(top_wishlist['title'], top_wishlist['total_wishlist'])
plt.title("Top 10 Most Wishlisted Games")
plt.gca().invert_yaxis()
plt.show()

#### 📝 Insights:
- ##### Wishlist rankings suggest strong predictive potential for commercial performance, as high pre-release interest often translates into higher launch sales.

**5.What’s the average plays per genre?**

In [ ]:
plays_by_genre = (
    games_df.groupby('genre')['plays']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure()
plays_by_genre.plot(kind='bar')
plt.title("Average Plays per Genre")
plt.show()


#### 📝 Insights:
- ##### MOBA games exhibit the highest average play counts, indicating extremely high engagement intensity and long-term player retention.
- ##### Shooter and Racing genres also demonstrate strong engagement levels, likely due to competitive gameplay mechanics and multiplayer ecosystems.
- ##### Tactical and Simulator genres show relatively lower average play counts, indicating niche audiences or more specialized gameplay experiences.

**6.Which region generates the most sales?**

In [ ]:
regional_sales = vgsales_df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum()
regional_percent = regional_sales / regional_sales.sum() * 100

plt.figure()
plt.pie(
    regional_percent,
    labels=regional_percent.index,
    autopct='%1.1f%%'
)
plt.title("Global Sales Share by Region")
plt.show()

#### 📝 Insights:
- ##### North America is the largest contributor to global video game sales, indicating the region’s strong consumer purchasing power and dominant gaming culture.
- ##### Europe represents a strong secondary market, contributing substantially to global revenue, though still trailing behind North America.
- ##### Japan contributes meaningfully to global sales but at a smaller scale compared to Western markets, suggesting regional genre and platform specialization.

**7.What are the top-performing Genre + Region combinations?**

In [ ]:
genre_region = vgsales_df.groupby('genre')[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum()

plt.figure(figsize=(8,6))
sns.heatmap(genre_region, cmap="Blues")
plt.title("Genre vs Region Sales Heatmap")
plt.show()

#### 📝 Insights:
- ##### North America leads sales across nearly all major genres, particularly Action, Sports, and Shooter, reinforcing its position as the dominant revenue-driving region in the gaming industry.
- ##### Japan demonstrates a strong preference for Role-Playing games, indicating genre specialization and culturally driven gaming preferences.
- ##### Sports and Shooter genres generate significantly higher sales in Western markets compared to Japan, suggesting regional demand differences tied to cultural and entertainment preferences.
- ##### Action games exhibit broad international appeal, making them one of the most commercially viable and globally scalable genres.

**8.What are the best-selling platforms?**

In [ ]:
platform_sales = (
    vgsales_df.groupby('platform')['global_sales']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure()
platform_sales.plot(kind='bar')
plt.title("Top 10 Best-Selling Platforms")
plt.show()


#### 📝 Insight:
##### PS2, X360 & PS3 significantly outperform others in global sales, reflecting strong hardware adoption cycles and exclusive title ecosystems.

**9.What’s the trend of releases and sales over time?**

In [ ]:
trend = (
    vgsales_df.groupby('year')
    .agg(total_sales=('global_sales','sum'),
         total_games=('name','count'))
)

trend[['total_sales','total_games']].plot()
plt.title("Releases and Sales Over Time")
plt.show()

#### 📝 Insights:
- ##### The gaming industry experienced rapid expansion between the mid-1990s and late-2000s, with both game production and total sales rising significantly. This period likely reflects the rise of major console generations and growing global adoption.
- ##### The industry reached its peak around 2008–2009 in both output and commercial performance, indicating a mature console market cycle and strong consumer demand.
- ##### Sales and number of games are strongly correlated. When releases increase → sales increase
- ##### The post-2010 decline may partially reflect limitations of the dataset, as it does not fully capture digital distribution, mobile gaming, and subscription-based revenue models.

**10.Top 10 best-selling games globally**

In [ ]:
top_games = (
    vgsales_df.groupby('name')['global_sales']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure()
top_games.plot(kind='bar')
plt.title("Top 10 Best-Selling Games")
plt.show()


#### 📝 Insight:

##### Top 10 games contribute disproportionately to total industry sales, highlighting the blockbuster-driven nature of the gaming industry.

**11.Does rating affect global sales?**

In [ ]:
sns.scatterplot(
    x='rating',
    y='global_sales',
    data=merged_data
)
plt.title("Rating vs Global Sales")
plt.show()

#### 📝 Insights:

- ##### Most games cluster around mid-to-high rating ranges (3.5–4.5), indicating generally positive user reception. Extremely low-rated games are rare, suggesting quality filtering in the market.
- ##### While higher-rated games generally show better sales performance, the relationship is moderate rather than absolute. Marketing strength and franchise reputation also influence sales beyond ratings.

**12.Do highly wishlisted games lead to more sales?**

In [ ]:
sns.scatterplot(
    x='wishlist',
    y='global_sales',
    data=merged_data
)
plt.xscale('log')
plt.yscale('log')
plt.title("Wishlist vs Sales")
plt.show()

sns.regplot(
    x='wishlist',
    y='global_sales',
    data=merged_data,
    scatter_kws={'alpha':0.5}
)
plt.xscale('log')
plt.yscale('log')
plt.show()

#### 📝 Insights:

- ##### Wishlist count appears to have a positive association with global sales, but the relationship is not strongly linear. High wishlist does not always guarantee high sales.
- ##### Most games are clustered below 10 million sales. Only a few extreme outliers reach 30+ or 80+ million.
- ##### The log-scaled scatter plot reveals a moderate positive association between wishlist counts and global sales. While higher wishlist values generally correspond to higher sales, significant dispersion indicates that wishlist alone is insufficient to predict commercial success. The distribution also reflects the blockbuster-driven nature of the gaming industry, where a small subset of titles dominate total revenue.

**13.What are the top-performing Genre + Platform combinations?**

In [ ]:
pivot = vgsales_df.pivot_table(
    values='global_sales',
    index='genre',
    columns='platform_category',
    aggfunc='sum'
)

plt.figure(figsize=(12,8))
sns.heatmap(pivot, cmap="BuGn")
plt.title("Genre-Platform Sales Heatmap")
plt.show()

#### 📝 Insights:

- ##### Maximum games are played on Play Station and Nintendo, followed by Xbox.
- ##### PlayStation Dominates Across Multiple Genres. It consistently shows the highest sales intensity across major genres such as Action, Sports, Shooter, and Racing. This indicates strong market penetration and broad genre appeal on Sony’s platform ecosystem.
- ##### Nintendo show particularly strong performance in Platform, Misc and Action genres, reinforcing Nintendo’s reputation for character-driven and franchise-based games.
- ##### Action & Sports games show peak performance on PlayStation, indicating a strong alignment between sports gaming audiences and Sony’s user base.
- ##### The Action genre demonstrates high sales across nearly all major platforms, reinforcing its position as the most commercially viable genre category.
- ##### Puzzle, Strategy & Simulation genres exhibit moderate but consistent sales across platforms, indicating steady but non-blockbuster demand.
- ##### Role-Playing games perform well across both Nintendo and PlayStation, suggesting broader demographic appeal compared to niche genres like Strategy or Puzzle.
- ##### Platforms such as Atari, NEC, Bandai, SNK and Panasonic display significantly lower genre-wise sales intensity, reflecting earlier console generations with smaller market scale.

**14.Which region generates more revenue?**

In [ ]:
region_trend = vgsales_df.groupby('year')[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum()

plt.figure()
region_trend.plot()
plt.title("Regional Sales Trend Over Time")
plt.xlabel("Year")
plt.ylabel("Total Sales")
plt.show()

#### 📝 Insights:
- ##### North America consistently leads global sales and drives overall industry peaks, particularly during the 2007–2009 console generation boom. Market fluctuations in NA strongly influence global performance trends.
- ##### Europe exhibits a similar growth trajectory to North America but at a smaller scale, suggesting aligned market behavior across Western regions with slightly delayed peak intensity.
- ##### Japan’s sales peak earlier and at a lower magnitude compared to Western markets, reflecting different console preferences and potentially earlier market saturation.
- ##### Emerging markets demonstrate steady growth leading up to the late-2000s peak, indicating gradual global expansion beyond traditional gaming regions.
- ##### The synchronized peak across regions suggests a strong global console generation effect, likely driven by the PS3, Xbox 360, and Wii era.
- ##### The post-2010 decline across all regions likely reflects console transition periods and the growing shift toward digital, mobile, and subscription-based gaming not fully captured in the dataset.

## **Solutions to the Business Objective**

- **Prioritize High-Performing Genres:**
Invest more resources in genres such as Action, RPG, and Shooter, which consistently demonstrate strong sales and engagement. These genres offer higher commercial scalability across multiple regions.
- **Leverage Wishlist as a Demand Forecasting Tool:**
Use wishlist counts as an early indicator of market demand to guide production scaling, marketing intensity, and launch strategy. Games with high pre-release interest should receive stronger promotional support and optimized launch timing.
- **Adopt Region-Specific Marketing Strategies:**
Tailor marketing and distribution strategies according to regional preferences—e.g., emphasize RPG titles in Japan and Shooter/Sports titles in North America and Europe.
- **Optimize Platform Targeting:**
Focus on platforms that historically generate higher sales within specific genres. Align genre development with platform strengths to maximize return on investment.
- **Balance Blockbuster and Niche Portfolio:**
While blockbuster franchises drive significant revenue, maintaining a diversified portfolio—including indie and niche titles—reduces risk and captures varied audience segments.
- **Enhance Engagement-Driven Monetization Models:**
For high-engagement genres (e.g., MOBA and Shooter), consider live-service models, expansions, and in-game monetization strategies to sustain long-term revenue.
- **Monitor Console Lifecycle Trends:**
Align major releases with peak console adoption cycles and prepare adaptive strategies during transitional periods between hardware generations. 

## **Conclusion**

#### This project provides a comprehensive analysis of the video game industry by integrating user engagement data with global and regional sales performance. The findings reveal that commercial success is influenced by a combination of genre popularity, regional market dynamics, platform strength, and pre-release engagement indicators such as wishlist counts. While blockbuster titles and established franchises dominate revenue generation, engagement intensity varies significantly across genres, highlighting differences between replay-driven and narrative-driven game models. The industry exhibits clear lifecycle patterns, with peak performance aligning with major console generations and subsequent shifts reflecting structural changes in distribution and consumption models. Overall, the analysis demonstrates that combining behavioral engagement metrics with sales data offers a deeper, data-driven understanding of what drives success in the gaming market, enabling more informed strategic decisions for developers and publishers.